In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import json
import matplotlib.pyplot as plt
import plotly.express as px
import imageio
import warnings
warnings.filterwarnings('ignore')


Bad key "text.kerning_factor" on line 4 in
C:\Users\akgoel\.conda\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
#Read in the data from the raw Zillow CSV for ZALL
zillow_path = Path("../data/clean_data/zall_metro.csv")
zillow_df = pd.read_csv(zillow_path, infer_datetime_format=True, parse_dates=True)

zillow_df["date"] = pd.to_datetime(zillow_df["date"])
zillow_df["region"] = zillow_df["region"].str.replace(", TX", "", regex=False)
zillow_year_df = zillow_df.groupby(["region",zillow_df["date"].dt.year]).mean()
zillow_year_df = zillow_year_df.reset_index()
zillow_year_df = zillow_year_df.rename(columns={"date":"YEAR"})
zillow_year_df.head()

,region,YEAR,avg_value
0,Austin,1996,165680.250000
1,Austin,1997,168675.916667
2,Austin,1998,172097.333333
3,Austin,1999,173058.083333
4,Austin,2000,187038.750000


In [9]:
#Read in the salary data from the IRS csv
salary_df = pd.read_csv("../data/clean_data/cnc_sal_data.csv")
salary_df.head()

,YEAR,ZIP CODE,NO. OF RETURNS,AGI
0,2007,73301,1798,7230
1,2007,73949,43,1915
2,2007,75001,7128,693056
3,2007,75002,26669,2002247
4,2007,75006,21953,1053748


In [10]:
zip_metro_df = pd.read_csv("../data/clean_data/zipcode_metro.csv")
zip_metro_df = zip_metro_df.rename(columns={"ZIPCODE" : "ZIP CODE", "ZONE":"region"})
zip_metro_df.head()

,ZIP CODE,region
0,77002,Houston
1,77003,Houston
2,77004,Houston
3,77005,Houston
4,77006,Houston


In [11]:
#Merge the salary data with the zip_metro data
salary_metro_df = pd.merge(zip_metro_df, salary_df, on=['ZIP CODE']).dropna()
salary_metro_df.head()

,ZIP CODE,region,YEAR,NO. OF RETURNS,AGI
0,77002,Houston,2007,3370,819744
1,77002,Houston,2008,3599,1259964
2,77002,Houston,2009,3816,978793
3,77002,Houston,2010,3952,1996053
4,77002,Houston,2011,4562,1595344


In [13]:
#Group by region
salary_metro_df = salary_metro_df.groupby(["region","YEAR"]).sum()
salary_metro_df["Avg_AGI"] = salary_metro_df["AGI"] / salary_metro_df["NO. OF RETURNS"] * 1000
salary_metro_df = salary_metro_df.reset_index()
salary_metro_df.head()

,region,YEAR,ZIP CODE,NO. OF RETURNS,AGI,Avg_AGI
0,Austin,2007,3306167,472473,32099099,67938.483257
1,Austin,2008,3306167,465682,30769974,66075.076984
2,Austin,2009,3306167,479563,30456486,63508.831999
3,Austin,2010,3306167,495992,33362017,67263.215939
4,Austin,2011,3306167,516754,36237502,70125.247216


In [16]:
#Merge the Zillow data with the salary data
combined_df = pd.merge(salary_metro_df, zillow_year_df, on=['region','YEAR'])
combined_df["OP_RATIO"] = combined_df['Avg_AGI'] / combined_df["avg_value"]
combined_df = combined_df.drop(columns=["AGI","NO. OF RETURNS","Avg_AGI","avg_value","ZIP CODE"])
combined_df

,region,YEAR,OP_RATIO
0,Austin,2007,0.313019
1,Austin,2008,0.300600
2,Austin,2009,0.299075
3,Austin,2010,0.320165
4,Austin,2011,0.344361
5,Austin,2012,0.393766
6,Austin,2013,0.343143
7,Austin,2014,0.332265
8,Austin,2015,0.322392
9,Austin,2016,0.294151
